In [29]:
import json

import requests
import sqlite3
import pandas as pd
import MeCab

In [2]:
conn = sqlite3.connect('./db/test.sqlite3')
c = conn.cursor()

# ポイント予測に必要なDataFrameの前処理

In [31]:
def preprocessing(text):
    text = re.sub('\n', '', text)
    text = re.sub('\r', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)
    return text

In [32]:
def count_noun_number(mecab, text):
    text = str(text)
    c = []
    for line in mecab.parse(text).splitlines():
        try:
            if "名詞" in line.split()[-1]:
                c.append(line)
        except:
            pass
    return len(set(c))

In [110]:
def point_prediction(url, conn, cursor, mode, ncodes=None):
    if mode not in ['all', 'part']:
        raise Exception('Argument mode shoud be all or part.')
    if mode == 'part' and type(ncodes) is not list:
        raise Exception('Argument ncodes should be list of string.')
    
    mecab = MeCab.Tagger("-Ochasen")
    
    if mode == 'all':
        detail_df = pd.read_sql_query(sql="SELECT * FROM details WHERE predict_point='Nan' LIMIT 10", con=conn)
        ncodes = list(detail_df.ncode)
    elif mode == 'part':
        ncodes_str = ', '.join(map(str, list(["'{}'".format(ncode) for ncode in ncodes])))
        detail_df = pd.read_sql_query(sql="SELECT * FROM details WHERE ncode IN ({})".format(ncodes_str), con=conn)
    
    for column in ['title', 'story', 'text']:
        detail_df[column + '_length'] = detail_df[column].apply(lambda x: len(str(x)))
    detail_df['keyword_number'] = detail_df['keyword'].apply(lambda x: len(str(x).split(' ')))
    detail_df['noun_proportion_in_text'] = detail_df.text.apply(lambda x: count_noun_number(mecab, str(x)) / len(str(x)))
    
    headers = {'Content-Type': 'application/json'}
    data = {}
    data = {column: list(detail_df[column]) for column in list(detail_df.columns)}
    data = json.dumps(data)
    r_post = requests.post(url, headers=headers, json=data)
    
    predicted_points = r_post.json()['prediction']
    ncodes = list(detail_df.ncode)
    for ncode, predicted_point in zip(ncodes, predicted_points):
        c.execute("UPDATE details SET predict_point={} WHERE ncode='{}'".format(predicted_point, ncode))

In [111]:
url = 'http://localhost:5000/predict'

In [112]:
point_prediction(url, conn, c, mode='all')

In [67]:
#c.execute("CREATE INDEX ncodeindex ON details(ncode)")

# カラム名取得

In [ ]:
c.execute('select * from details limit 1')

In [74]:
names = list(map(lambda x: x[0], c.description))

In [75]:
names

['index',
 'title',
 'ncode',
 'userid',
 'writer',
 'story',
 'biggenre',
 'genre',
 'keyword',
 'general_firstup',
 'general_lastup',
 'novel_type',
 'end',
 'general_all_no',
 'length',
 'time',
 'isstop',
 'isr15',
 'isbl',
 'isgl',
 'iszankoku',
 'istensei',
 'istenni',
 'pc_or_k',
 'global_point',
 'daily_point',
 'weekly_point',
 'monthly_point',
 'quarter_point',
 'yearly_point',
 'fav_novel_cnt',
 'impression_cnt',
 'review_cnt',
 'all_point',
 'all_hyoka_cnt',
 'sasie_cnt',
 'kaiwaritu',
 'novelupdated_at',
 'updated_at',
 'weekly_unique',
 'text',
 'predict_point']